# Kaggle Competition - Logistic Classification

In [1]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import urllib.request

from IPython.core.interactiveshell import InteractiveShell
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsOneClassifier
from sklearn.preprocessing import StandardScaler

InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [3]:
# load data
x = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/train_features'))
test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
y = np.array(pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/train_labels')))
y = y.astype(float)

# load transformed image data
color_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/color_features'))
compress_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/compress_features'))
crop_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/crop_features'))
crop_to_corner_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/crop_to_corner_features'))
homography_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/homography_features'))
mirror_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/mirror_features'))
rotate30_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/rotate30_features'))
scale_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/scale_features'))

# subset the data (if wanting to use a smaller number of classes)
classes = np.unique(y)
index = np.ravel(np.nonzero(np.in1d(y, classes)))
x_subset = x[index]
y_subset = y[index]

In [ ]:
def computeobj(beta, lamb, x, y):
    # number of observations
    n = x.shape[0]
    
    # compute objective function
    obj = (1/n)*(np.sum(np.log(1 + np.exp(-y*np.dot(x, beta))))) + lamb*np.sum(beta**2)
    
    return obj


def computegrad(beta, lamb, x, y):
    # number of observations
    n = x.shape[0]
    
    # compute gradient of objective function
    grad_beta = -(1/n)*(np.dot(x.T, y/(np.exp(y*np.dot(x, beta)) + 1))) + 2*lamb*beta
    
    return grad_beta


def backtracking(beta, lamb, x, y, eta=1, alpha=0.5, gamma=0.8, max_iter=100):
    # initialize variables
    grad_beta = computegrad(beta, lamb, x, y)
    norm_grad_beta = np.sqrt(np.sum(grad_beta**2))
    found_eta = 0
    t = 0
    
    # loop through until eta found or max iterations reached
    while found_eta == 0 and t < max_iter:
        if (computeobj(beta - eta*grad_beta, lamb, x, y) <
                computeobj(beta, lamb, x, y) - alpha*eta*norm_grad_beta**2):
            found_eta = 1
        elif t == max_iter:
            break
        else:
            eta = eta*gamma
            t += 1
    
    return eta


def fastgradalgo(beta_init, theta_init, lamb, x, y, max_iter, eps):
    # initialize variables
    beta = beta_init
    theta = theta_init
    grad_theta = computegrad(theta, lamb, x, y)
    eta_init = 1/(max(np.linalg.eigh(np.dot((1/n)*x.T, x))[0]) + lamb)
    beta_vals = [beta_init]
    t = 0
    
    # loop through until EITHER max iterations reached or threshold of epsilon reached
    while t < max_iter and np.linalg.norm(grad_theta) >  eps:
        eta = backtracking(beta, lamb, x, y, eta=eta_init)
        beta_next = theta - eta*grad_theta
        theta = beta_next + t*(beta_next - beta)/(t + 3)
        grad_theta = computegrad(theta, lamb, x, y)
        beta = beta_next
        beta_vals.append(beta)
        t += 1
        
    return beta_vals


def split_data_equal(x, y, test_set, train_size=0.75):
    # split into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0, stratify=y)
    
    # center and standardize x values
    x_scaler = StandardScaler().fit(x_train)
    x_train = x_scaler.transform(x_train)
    x_test = x_scaler.transform(x_test)
    test_set = x_scaler.transform(test_set)
    
    return x_train, x_test, y_train, y_test, test_set


def train_alg(x, y, classes, lamb_list):
    # initialize betas and create list for final beta values
    d = x.shape[1]
    beta_init = np.zeros(d)
    theta_init = np.zeros(d)
    final_betas = []
    y_values = []

    # loop through each label and perform ovr appending the final betas for each class
    for i in range(len(classes)):
        lamb = lamb_list[i]
        y_binary = copy.deepcopy(y)
        y_binary[y != classes[i]] = -1
        y_binary[y == classes[i]] = 1
        betas = fastgradalgo(beta_init=beta_init, theta_init=theta_init, lamb=lamb, x=x_train, y=y_binary, max_iter=1000, eps=1e-5)[-1]
        final_betas.append(betas)
        
    return np.array(final_betas)


def predict(x, betas, classes):
    # initialize calculated y values
    y_values = []
    
    # loop through set of final betas and calculate y values
    for i in range(len(betas)):
        y_values.append(np.dot(x, betas[i]))
    
    # calculate predicted values
    y_predict = classes[np.argmax(np.array(y_values), axis=0)]
    
    return y_predict


def accuracy_misclass_error(predict, actual):
    # calculate misclassification error
    misclass_error = np.mean(predict != actual)*100
    accuracy = 100 - misclass_error
    
    return accuracy, misclass_error


def display_confusion_matrix(predict, actual):
    # calculate confusion matrix
    conf_mat = confusion_matrix(y_true=actual, y_pred=predict)
    
    # build visual plot
    plt.matshow(conf_mat);
    plt.title('Confusion Matrix');
    plt.xlabel('Predicted Label');
    plt.ylabel('True Label');
    plt.xticks(range(len(classes)), classes);
    plt.yticks(range(len(classes)), classes);
    
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 18
    fig_size[1] = 12
    plt.rcParams["figure.figsize"] = fig_size
    
    return conf_mat
    

def class_diff(classes, conf_matrix):
    # initialize variable to append each individual class percent
    percent_correct = []
    
    # loop through confusion matrix by true label
    for i in range(len(conf_matrix[0, :])):
        class_count = np.sum(conf_matrix[i])
        misclass_count = 0
        
        # loop through confusion matrix by predict label and append percent correct
        for j in range(len(conf_matrix[:, 0])):
            if i != j:
                misclass_count += conf_matrix[i][j]
            else:
                pass
        percent_correct.append(misclass_count/class_count)
        
    # calcuate ordered list of multi-class misclassification error
    ordered_class_diff = np.vstack((classes, np.array(percent_correct))).T
    ordered_class_diff = ordered_class_diff[ordered_class_diff[:, 1].argsort()[::-1]]
    
    return ordered_class_diff


def export_csv(y, name):
    y = y.astype(int)
    df = pd.DataFrame({'Id':np.arange(1, y.shape[0] + 1), 'Prediction':y})
    df.to_csv(name, sep=',', index=False)
    

def decomp_PCA(train, test, test_set, explained_var_threshold=0.95):
    pca = PCA().fit(train)

    pca_explained_var_ratio = pca.explained_variance_ratio_

    pca_explained_var = []
    num_component_vectors = 0

    while np.sum(pca_explained_var) < explained_var_threshold:
        pca_explained_var.append(pca_explained_var_ratio[num_component_vectors])
        num_component_vectors += 1
    #print('# Component Vectors: %d    Explained Var: %f' % (num_component_vectors, np.sum(pca_explained_var)))

    pca = PCA(n_components=num_component_vectors).fit(train)
    x_train = pca.transform(train)
    x_test = pca.transform(test)
    test_set = pca.transform(test_set)
    
    return x_train, x_test, test_set

## Running my own logistic one-vs-rest and comparing to sklearn's LogisticRegression

In [4]:
# split data
test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
x_train, x_test, y_train, y_test, test_set = split_data_equal(x_subset, y_subset, test_set)
n = x_train.shape[0]
d = x_train.shape[1]

In [5]:
# run PCA to reduce dimensionality
x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)

In [6]:
lamb_list = np.ones(x_train.shape[1]).tolist()
betas = train_alg(x=x_train, y=y_train, classes=classes, lamb_list=lamb_list)
y_predict = predict(x=x_test, betas=betas, classes=classes)
accuracy, misclass_error = accuracy_misclass_error(predict=y_predict, actual=y_test)

In [7]:
print('Accuracy: %f%%' % (accuracy))
print('Misclassification Error: %f%%' % (misclass_error))

Accuracy: 58.055556%
Misclassification Error: 41.944444%


In [8]:
pd.DataFrame(betas)

,0,1,2,3,4,5,6,7,8,9,...,546,547,548,549,550,551,552,553,554,555
0,0.004136,-0.002433,-0.000428,0.000565,-0.002156,0.003469,-0.002282,-0.000721,-0.000386,0.000127,...,-0.000527,0.000286,-0.000047,-0.000443,0.000253,-0.000418,0.000260,0.000235,0.000077,-1.890328e-06
1,0.003709,-0.003788,0.000416,0.002111,-0.003284,0.004385,-0.003719,-0.002167,-0.000315,-0.002996,...,0.000301,0.000145,0.000262,0.000541,0.000081,0.000820,0.000184,-0.000442,-0.000268,1.002672e-04
2,0.000765,0.003829,0.002299,0.004581,-0.000053,0.002981,0.004929,0.000662,0.002615,-0.006728,...,-0.000731,0.000113,0.000858,-0.000550,-0.000328,0.000638,0.000123,0.000094,0.000231,6.112033e-04
3,0.000354,0.003069,0.001730,0.002458,-0.000930,-0.000292,0.001924,-0.001972,0.003794,-0.006610,...,0.000191,0.000084,-0.000732,0.000647,0.000053,0.000692,0.000164,-0.000271,-0.000326,3.995256e-04
4,-0.000720,0.001094,-0.001691,0.002049,-0.000633,-0.001060,0.000246,-0.002912,-0.000490,-0.002655,...,-0.000258,0.000013,0.000485,0.000258,-0.000186,-0.000565,0.000163,-0.000141,0.000090,-1.342996e-04
5,-0.000385,0.002233,0.001528,0.000794,-0.001274,0.000016,0.000184,0.000261,0.002355,-0.001920,...,-0.000542,-0.000162,0.000223,-0.000329,-0.000106,-0.000101,-0.000418,-0.000757,0.000074,-2.758214e-04
6,-0.000735,-0.003866,0.001822,0.004793,-0.002174,0.001035,0.008098,-0.010007,0.001705,0.006933,...,-0.000053,-0.000259,0.000479,0.000272,-0.000141,-0.000024,-0.000414,0.000384,0.000271,2.495894e-04
7,-0.001278,0.001247,-0.000261,0.001084,-0.000781,-0.002028,-0.000210,-0.003072,0.004633,-0.005552,...,-0.000190,0.000380,-0.000170,0.000460,0.000576,-0.000820,-0.000798,-0.000218,0.000605,-9.708839e-04
8,0.003628,0.002498,0.000671,-0.000938,0.001455,0.001305,0.002920,-0.000362,0.003606,-0.000247,...,0.000068,-0.000208,-0.000027,-0.000629,-0.000141,-0.000071,-0.000433,0.000018,0.000441,-1.010286e-05
9,0.000353,0.003059,0.000898,0.003030,-0.001898,-0.001303,0.003277,-0.003408,0.003455,-0.003666,...,0.000076,0.000003,-0.000364,0.000136,0.000265,0.000413,-0.000656,-0.000455,-0.000244,-4.700317e-04


In [9]:
logitCV = LogisticRegression(C=1/(2*n*1), fit_intercept=False).fit(x_train, y_train)
betas = logitCV.coef_
y_predict = predict(x=x_test, betas=betas, classes=classes)
accuracy, misclass_error = accuracy_misclass_error(predict=y_predict, actual=y_test)

In [10]:
print('Accuracy: %f%%' % (accuracy))
print('Misclassification Error: %f%%' % (misclass_error))

Accuracy: 58.055556%
Misclassification Error: 41.944444%


In [11]:
pd.DataFrame(betas)

,0,1,2,3,4,5,6,7,8,9,...,546,547,548,549,550,551,552,553,554,555
0,0.004136,-0.002433,-0.000428,0.000565,-0.002156,0.003469,-0.002282,-0.000721,-0.000386,0.000127,...,-0.000528,0.000286,-0.000047,-0.000443,0.000254,-0.000418,0.000260,0.000235,0.000077,-1.884837e-06
1,0.003709,-0.003788,0.000416,0.002111,-0.003284,0.004385,-0.003719,-0.002167,-0.000315,-0.002996,...,0.000301,0.000145,0.000262,0.000541,0.000081,0.000820,0.000184,-0.000442,-0.000268,1.003011e-04
2,0.000765,0.003829,0.002299,0.004581,-0.000053,0.002981,0.004929,0.000662,0.002615,-0.006728,...,-0.000731,0.000113,0.000858,-0.000551,-0.000328,0.000638,0.000123,0.000094,0.000231,6.114488e-04
3,0.000354,0.003069,0.001730,0.002458,-0.000930,-0.000292,0.001924,-0.001972,0.003794,-0.006610,...,0.000191,0.000084,-0.000733,0.000647,0.000053,0.000692,0.000164,-0.000271,-0.000326,3.996506e-04
4,-0.000720,0.001094,-0.001691,0.002049,-0.000633,-0.001060,0.000246,-0.002912,-0.000490,-0.002655,...,-0.000258,0.000013,0.000485,0.000258,-0.000186,-0.000565,0.000163,-0.000140,0.000090,-1.342316e-04
5,-0.000385,0.002233,0.001528,0.000794,-0.001274,0.000016,0.000184,0.000261,0.002355,-0.001920,...,-0.000541,-0.000162,0.000223,-0.000329,-0.000106,-0.000101,-0.000418,-0.000757,0.000074,-2.756922e-04
6,-0.000735,-0.003866,0.001822,0.004793,-0.002174,0.001035,0.008098,-0.010007,0.001705,0.006933,...,-0.000053,-0.000259,0.000480,0.000272,-0.000141,-0.000024,-0.000414,0.000384,0.000271,2.497173e-04
7,-0.001278,0.001247,-0.000261,0.001084,-0.000781,-0.002028,-0.000210,-0.003072,0.004633,-0.005552,...,-0.000190,0.000380,-0.000170,0.000460,0.000575,-0.000820,-0.000798,-0.000218,0.000605,-9.704291e-04
8,0.003628,0.002498,0.000671,-0.000938,0.001455,0.001305,0.002920,-0.000362,0.003606,-0.000247,...,0.000068,-0.000208,-0.000027,-0.000629,-0.000141,-0.000071,-0.000433,0.000018,0.000441,-1.010135e-05
9,0.000353,0.003059,0.000898,0.003030,-0.001898,-0.001303,0.003277,-0.003408,0.003455,-0.003666,...,0.000076,0.000003,-0.000364,0.000136,0.000265,0.000413,-0.000656,-0.000455,-0.000244,-4.702065e-04


## Running sklearn's LogisticRegressionCV for improved performance

In [12]:
# try stacking image tansformations onto the original features
allfeatures = np.vstack((x))
alllabels = np.hstack((y))
test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))

In [13]:
# split data
x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
n = x_train.shape[0]
d = x_train.shape[1]

In [14]:
# run PCA to reduce dimensionality
x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)

In [15]:
'''
Fitting various models is time intensive, especially when trying out multiple sets of features.
For this reason, the code below has been commented out and the allfeatures and alllabels are
set statically at what they were last processed as. Pickle files were used to save the models
and are called in the last cell to show results.
'''

# one-vs-one
# ovo_logit_estimator = LogisticRegressionCV(fit_intercept=False, dual=False, n_jobs=-1)
# ovo_logit = OneVsOneClassifier(ovo_logit_estimator, n_jobs=-1).fit(x_train, y_train)

# one-vs-rest
# ovr_logit = LogisticRegressionCV(fit_intercept=False, dual=False, n_jobs=-1).fit(x_train, y_train)

# multinomial
# mult_logit = LogisticRegressionCV(fit_intercept=False, dual=False, multi_class='multinomial', n_jobs=-1).fit(x_train, y_train)

'\nFitting various models is time intensive, especially when trying out multiple sets of features.\nFor this reason, the code below has been commented out and the allfeatures and alllabels are\nset statically at what they were last processed as. Pickle files were used to save the models\nand are called in the last cell to show results.\n'

In [16]:
# y_predict_ovo = ovo_logit.predict(x_test)
# y_predict_ovr = ovr_logit.predict(x_test)
# y_predict_mult = mult_logit.predict(x_test)
# print('Accuracy: %f%%' % (np.mean(y_predict_ovo == y_test)*100))
# print('Accuracy: %f%%' % (np.mean(y_predict_ovr == y_test)*100))
# print('Accuracy: %f%%' % (np.mean(y_predict_mult == y_test)*100))

## Aggregation of results

In [22]:
# for purposes of efficiency, the models were saved using pickle
all_test_set_predictions_header = ['Id']
all_test_set_predictions = np.arange(1, 4321)

# declare a dictionary for models
my_dict = {}

# image transformations feature sets
feature_sets = ['base',
                'color',
                'compress',
                'crop',
                'crop_to_corner',
                'homography',
                'mirror',
                'rotate30',
                'scale']

#### one-vs-one - due to the size of the pickle files for one-vs-one fitted models (approximately 1.5Gb), these have been commented out. Should you like to run, them, you may run the one-vs-one above for each stack of data.

In [26]:
# ovo - print all past results for each image transformation added to the base set
# print('one-vs-one\nadding image transformations 1 at a time')
# for i in feature_sets:
#     my_dict['ovo_logitCV_{0}'.format(i)] = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/LogisticRegressionCV/ovo_logitCV_' + i))
#     if i == 'base':
#         allfeatures = x
#         alllabels = y
#     else:
#         allfeatures = np.vstack((x, globals()[i + '_set']))
#         alllabels = np.hstack((y, y))
#     test_set = test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
#     x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
#     x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)
#     all_test_set_predictions_header = all_test_set_predictions_header + ['ovo_' + i]
#     all_test_set_predictions = np.vstack((all_test_set_predictions, my_dict['ovo_logitCV_' + i].predict(test_set)))
#     print(i, ': %f%%' % (np.mean(my_dict['ovo_logitCV_' + i].predict(x_test) == y_test)*100))

one-vs-one
adding image transformations 1 at a time
base : 65.648148%
color : 62.407407%
compress : 54.537037%
crop : 77.870370%
crop_to_corner : 61.111111%
homography : 82.129630%
mirror : 84.027778%
rotate30 : 46.574074%
scale : 63.055556%


#### one-vs-rest

In [23]:
# ovr - print all past results for each image transformation added to the base set
print('one-vs-rest\nadding image transformations 1 at a time')
for i in feature_sets:
    my_dict['ovr_logitCV_{0}'.format(i)] = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/LogisticRegressionCV/ovr_logitCV_' + i))
    if i == 'base':
        allfeatures = x
        alllabels = y
    else:
        allfeatures = np.vstack((x, globals()[i + '_set']))
        alllabels = np.hstack((y, y))
    test_set = test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
    x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
    x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)
    all_test_set_predictions_header = all_test_set_predictions_header + ['ovr_' + i]
    all_test_set_predictions = np.vstack((all_test_set_predictions, my_dict['ovr_logitCV_' + i].predict(test_set)))
    print(i, ': %f%%' % (np.mean(my_dict['ovr_logitCV_' + i].predict(x_test) == y_test)*100))

one-vs-rest
adding image transformations 1 at a time
base : 55.833333%
color : 50.462963%
compress : 46.111111%
crop : 54.490741%
crop_to_corner : 41.666667%
homography : 55.648148%
mirror : 56.111111%
rotate30 : 38.750000%
scale : 50.370370%


#### multinomial

In [24]:
# multinomial - print all past results for each image transformation added to the base set
print('multinomial\nadding image transformations 1 at a time')
for i in feature_sets:
    my_dict['mult_logitCV_{0}'.format(i)] = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/LogisticRegressionCV/mult_logitCV_' + i))
    if i == 'base':
        allfeatures = x
        alllabels = y
    else:
        allfeatures = np.vstack((x, globals()[i + '_set']))
        alllabels = np.hstack((y, y))
    test_set = test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
    x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
    x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)
    all_test_set_predictions_header = all_test_set_predictions_header + ['mult_' + i]
    all_test_set_predictions = np.vstack((all_test_set_predictions, my_dict['mult_logitCV_' + i].predict(test_set)))
    print(i, ': %f%%' % (np.mean(my_dict['mult_logitCV_' + i].predict(x_test) == y_test)*100))

multinomial
adding image transformations 1 at a time
base : 71.666667%
color : 66.574074%
compress : 58.101852%
crop : 83.518519%
crop_to_corner : 65.879630%
homography : 89.629630%
mirror : 91.712963%
rotate30 : 49.537037%
scale : 68.009259%


#### output all predictions

In [27]:
df = pd.DataFrame(all_test_set_predictions.astype(int).T)
df.to_csv('All_LogisticCV_Predictions.csv', sep=',', header=all_test_set_predictions_header, index=False)